In [1]:
import cv2
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
#from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image

2023-01-27 20:32:04.581085: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 20:32:04.700931: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/stk0057/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-27 20:32:04.700954: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-27 20:32:05.404371: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dler

In [2]:
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing import image

# Define the base directory
base_dir = "AIOralCancerProject"

# Create an empty list to store the images and labels
X = []
y = []

# Get the class labels from the directory names
class_labels = os.listdir(base_dir)

# Iterate through each directory and store the images and labels
for i, label in enumerate(class_labels):
    img_dir = os.path.join(base_dir, label)
    for file in os.listdir(img_dir):
        # Check if file is an image
        if file.endswith(".jpg") or file.endswith(".png"):
            # Load the image
            
            img = Image.open(os.path.join(img_dir, file))
            img = img.resize((300, 300))
            img = np.array(img)
            #img = img / 255.0

            # Append the image to the list of images
            X.append(img)
            # Append the corresponding label to the list of labels
            y.append(i)

In [3]:
# Shuffle the data
#X, y = shuffle(X, y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(-1,300,300,3)
X_test = X_test.reshape(-1,300,300,3)

In [5]:
output = []
for x in y_test:
    if x not in output:
        output.append(x)
print(output)

[9, 6, 4, 3, 14, 15, 11, 1, 5, 12, 2, 8, 16, 13, 0, 7]


In [6]:
X_train.shape

(2224, 300, 300, 3)

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
# Define the data generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [9]:
# Fit the generator to the training data
datagen.fit(X_train)

In [13]:
# Make sure y_train and y_test are in the correct format
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [11]:
from keras.utils import to_categorical

# One-hot encode the labels
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [14]:
y_train[y_train >= 10] -= 10
y_test[y_test >= 10] -= 10

In [15]:
y_train = to_categorical(y_train, num_classes=16)
y_test = to_categorical(y_test, num_classes=16)

In [16]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# Create the base pre-trained model
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)

# Freeze the base model layers
base_model.trainable = False
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer
predictions = Dense(16, activation='softmax')(x)
#predictions = Dense(16, activation='softmax')(predictions)


# Create a new model on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dropout(0.5)(x)
predictions = Dense(16, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


# This is the model we will train
#dense_layer = Dense(1, activation='sigmoid')
#output = dense_layer(predictions)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_test, y_test))


########

2023-01-27 20:34:42.743405: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-27 20:34:42.743478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mistakenly-eager-jawfish): /proc/driver/nvidia/version does not exist
2023-01-27 20:34:42.744741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30
35/35 [==============================] - 131s 4s/step - loss: 2.1094 - accuracy: 0.2792 - val_loss: 1.8292 - val_accuracy: 0.3705
Epoch 2/30
35/35 [==============================] - 129s 4s/step - loss: 1.7098 - accuracy: 0.4015 - val_loss: 1.6671 - val_accuracy: 0.4227
Epoch 3/30
35/35 [==============================] - 120s 3s/step - loss: 1.5408 - accuracy: 0.4627 - val_loss: 1.5919 - val_accuracy: 0.4281
Epoch 4/30
35/35 [==============================] - 122s 4s/step - loss: 1.4570 - accuracy: 0.4951 - val_loss: 1.6067 - val_accuracy: 0.4388
Epoch 5/30
35/35 [==============================] - 120s 3s/step - loss: 1.3960 - accuracy: 0.5171 - val_loss: 1.5337 - val_accuracy: 0.4604
Epoch 6/30
35/35 [==============================] - 124s 4s/step - loss: 1.3113 - accuracy: 0.5517 - val_loss: 1.5322 - val_accuracy: 0.4622
Epoch 7/30
35/35 [==============================] - 129s 4s/step - loss: 1.2437 - accuracy: 0.5760 - val_loss: 1.5061 - val_accuracy: 0.4784
Epoch 8/30
35

In [ ]:
import tensorflow as tf
tf.saved_model.save(model, 'AIOralCancerProject')

In [19]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

18/18 [==============================] - 22s 1s/step - loss: 1.3758 - accuracy: 0.5000
Test accuracy: 0.5
